## Before Beginning

**Note that for notebooks** the multiprocessing start method should be set to `fork` (default on Linux).
To use with `spawn` (default on Windows and macOS), use the `multiprocess` library.

Lets check libEnsemble is installed. Pandas is also used in the Ax generator.

In [ ]:
!pip install libensemble
!pip install pandas

## Multi-fidelity optimization with Ax

This is an example of Bayesian optimization using a multi-fidelity workflow.

The Ax generator uses an MTGP (Multi-Task Gaussian Process), which tracks the correlation between high and low fidelity simulation outputs.

In [ ]:
import numpy as np
import pprint

from libensemble.libE import libE
from libensemble.message_numbers import WORKER_DONE
from libensemble.alloc_funcs.start_only_persistent import only_persistent_gens
from libensemble.gen_funcs.persistent_ax_multitask import persistent_gp_mt_ax_gen_f
from libensemble.tools import add_unique_random_streams, parse_args, save_libE_output

In [ ]:
def run_simulation(H, persis_info, sim_specs, libE_info):
    calc_status = 0
    
    # Extract input parameters
    values = list(H["x"][0])
    x0 = values[0]
    x1 = values[1]
    # Extract fidelity parameter
    task = H["task"][0]
    if task == "expensive_model":
        z = 8
    elif task == "cheap_model":
        z = 1

    libE_output = np.zeros(1, dtype=sim_specs["out"])
    calc_status = WORKER_DONE

    # Function that depends on the resolution parameter
    libE_output["f"] = -(x0 + 10 * np.cos(x0 + 0.1 * z)) * (x1 + 5 * np.cos(x1 - 0.2 * z))

    return libE_output, persis_info, calc_status

In [ ]:
if __name__ == "__main__":
    nworkers = 5
    libE_specs = {"nworkers": nworkers, "comms": "local"}

    mt_params = {
        "name_hifi": "expensive_model",
        "name_lofi": "cheap_model",
        "n_init_hifi": 4,
        "n_init_lofi": 4,
        "n_opt_hifi": 2,
        "n_opt_lofi": 4,
    }

    sim_specs = {
        "sim_f": run_simulation,
        "in": ["x", "task"],
        "out": [("f", float)],
    }

    gen_specs = {
        # Generator function. Will randomly generate new sim inputs 'x'.
        "gen_f": persistent_gp_mt_ax_gen_f,
        # Generator input. This is a RNG, no need for inputs.
        "in": ["sim_id", "x", "f", "task"],
        "persis_in": ["sim_id", "x", "f", "task"],
        "out": [
            # parameters to input into the simulation.
            ("x", float, (2,)),
            ("task", str, max([len(mt_params["name_hifi"]), len(mt_params["name_lofi"])])),
            ("resource_sets", int),
        ],
        "user": {
            "range": [1, 8],
            # Total max number of sims running concurrently.
            "gen_batch_size": nworkers - 1,
            # Lower bound for the n parameters.
            "lb": np.array([0, 0]),
            # Upper bound for the n parameters.
            "ub": np.array([15, 15]),
        },
    }
    gen_specs["user"] = {**gen_specs["user"], **mt_params}

    alloc_specs = {
        "alloc_f": only_persistent_gens,
        "user": {"async_return": False},
    }

    # Exit criteria
    exit_criteria = {"sim_max": 20}  # Exit after running sim_max simulations

    # Create a different random number stream for each worker and the manager
    persis_info = add_unique_random_streams({}, nworkers + 1)

    # Run LibEnsemble, and store results in history array H
    H, persis_info, flag = libE(
        sim_specs, gen_specs, exit_criteria, persis_info, alloc_specs, libE_specs
    )
    
    # Print 20 lines of input/output values
    pprint.pp(H[["x", "f"]][:20])

This is a simple example of a multi-fidelity workflow. Note that when using the executor to run applications, this can be combined with libEnsemble's dynamic resources feature to vary resources (nodes/cores/GPUs) given to each worker for each simulation.